### 动态规划
&emsp;&emsp;动态规划与分治法相似，都是通过组合子问题的解来求解原问题，分治法是将问题划分为互不相交的子问题，递归的求解子问题，再将他们的解组合起来，求出原问题的解。与之相反，动态规划应用于子问题重叠的情况，即不同的子问题具有公共的子子问题，在这种情况下，分治算法会做许多不必要的工作，它会反复的求解那些公共子问题，而动态规划算法对每个子子问题只求解一次，将其解保存在一个表格中，从而无需每次求解一个个子子问题时都重新计算，避免不必要的计算工作。

&emsp;&emsp;动态规划方法通常用来求解最优化问题，这类问题可以有很多解，每个解都有一个值，希望寻找具有最优值的解，称这样的解为问题的一个最优解。通常按照如下4个步骤来设计一个动态规划算法：

    1.刻画一个最优解的结构特征
    2.递归的定义最优解的值
    3.利用最优解的值，通常采用自底向上的方法
    4.利用计算出的信息构造一个最优解

#### 1.钢条切割

In [1]:
'''
    author;fengjiaxin
    desc:动态规划切钢条
    最优子结构性质：问题的最优解由相关的子问题的最优解组合而成，而这些子问题可以独立求解
    动态规划和分治法结构上相同，都是将问题分为小的子问题，然后合并
    但是分治法会出现一种情况，将小问题完全独立，但是有一些计算是重复的
    动态规划的出现就是将这些小问题的解记录下来，之后再遇到的时候可以直接调用，而不用直接计算
    相比分治法就是记录下来之前的解
'''

def recrusion_cut_rod(p,n):
    '''
        利用递归的方法解决钢条问题
    :param p:
    :param n:
    :return:
        返回最大利益
    '''
    if n == 0:
        return 0
    q = float('-Inf')
    for i in range(1,n+1):
        q = max(q,p[i] + recrusion_cut_rod(p,n-i))
    return q

'''
    可以看出，朴素递归的算法之所以效率低，是因为它反复的求解相同的子问题，因此动态规划算法仔细安排求解顺序
    对每个子问题只求解一次，并将结果存储下来，随后再次需要此子问题的解，只需查找保存的结果，不必重新计算
    动态规划是以空间换取时间的算法
    动态规划分为两种
    1.带备忘的自顶向下法，此方法按照自然的递归形式编写过程，但过程会保存每个子问题的结果，当需要一个子问题的解时，过程首先会检查是否已经保存过
        此解，如果是，直接返回保存的值，否则进行计算，这个过程是带备忘的，因为记住了之前的计算结果
    2.自底向上法，这个方法一般需要自定义子问题规模的概念，使得任何的子问题的求解都只依赖于更小的子问题的求解，可以将子问题按照规模排序，按由小到大的
        顺序进行求解，当求解某个子问题时，所依赖的更小的子问题都已经求解完毕，结果已经保存，每个子问题只需要求解一次

'''

# 带备忘机制的自顶向下的方法
def memoized_cut_rod(p,n):

    r = []
    for _ in range(0,n+1):
        r.append(float('-Inf'))
    return memoized_cut_rod_aux(p,n,r)

def memoized_cut_rod_aux(p,n,r):
    if r[n] >= 0:
        return r[n]
    if n == 0:
        q = 0
    else:
        q = float('-Inf')
        for i in range(1,n+1):
            q = max(q,p[i] + memoized_cut_rod_aux(p,n-i,r))
    r[n] = q
    return q

# 自顶向上的方法
def bottom_up_cut_rod(p,n):
    r = []
    for _ in range(0,n+1):
        r.append(float('-Inf'))
    r[0] = 0
    for j in range(1,n+1):
        q = float('-Inf')
        for i in range(1,j+1):
            q = max(q,p[i] + r[j-i])
        r[j] = q
    return r[n]

'''
    重构解:之前的钢条切割问题只是返回最优解的收益值，并未返回解本身（一个长度列表，给出切割后每段钢条的长度）
    它对长度为j的钢条不仅计算最大收益rj,还保存最优解对应的第一段钢条的切条长度sj
'''
def extended_bottom_up_cut_rod(p,n):
    r = []
    s = []
    for i in range(0,n + 1):
        r.append(float('-Inf'))
        s.append(float('-Inf'))
    r[0] = 0
    for j in range(1,n+1):
        q = float('-Inf')
        for i in range(1,j+1):
            if q < p[i] + r[j-i]:
                q = p[i] + r[j-i]
                s[j] = i
        r[j] = q
    return r,s


if __name__  == '__main__':
    parry = [0, 1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
    for k in range(1, 11):
        print(bottom_up_cut_rod(parry, k))

1
5
8
10
13
17
18
22
25
30


#### 2.动态规划原理
&emsp;&emsp;在本节，关注适用于动态规划方法求解的最优化问题应该具备的两个要素：最优子结构和子问题重叠。还会再次讨论备忘录方法，更加深入地讨论在自顶向下方法中如何借助备忘机制来充分利用子问题的重叠特性
##### 2.1 最优子结构
&emsp;&emsp;如果一个问题的最优解包含其子问题的最优解，称此问题具有最优子结构性质，在使用动态规划方法时，用子问题的最优解来构造原问题的最优解。可以发现，在发掘最优子结构性质的过程中，实际遵循了以下的通用模式：

    1.证明问题的最优解的第一个组成部分是作出一个选择，作出这次选择会产生一个或者多个带解的子问题
    2.对于一个给定问题，在其可能的第一步选择中，假定已经直到哪种选择才会得到最优解。
    3.在给定可获得最优解的选择后，确定这次选择会产生哪些子问题，以及如何最好地刻画子问题空间
    4.利用 剪切 - 粘贴 技术证明：作为构成原问题最优解的组成部分，每个子问题的解就是它本身的最优解

##### 2.2 重叠子问题
&emsp;&emsp;适合动态规划方法求解的最优化问题应该具备的第二个性质是子问题空间必须足够小，即问题的递归算法会反复的求解相同的子问题，而不是一直生成新的子问题，如果递归算法反复求解相同的子问题，就称最优化问题具有重叠子问题。与之相对的，适合用分治方法求解的问题通常在递归的每一步都生成全新的子问题，动态规划算法通常利用重叠子问题性质：对每个子问题求解一次，将解存入到一个表中，当再次需要这个子问题时直接查表，每次查找的代价为常量时间。

##### 2.3 备忘
&emsp;&emsp;可以保持自顶向下的策略，同时达到与自底向上动态规划方法相似的效率，思路就是对自然但低效的递归算法加入备忘机制，与自底向上方法一样，维护一个表记录子问题的解，但仍然保持递归算法的控制流程。

&emsp;&emsp;带备忘的递归算法为每个子问题维护一个表项来保存它的解，每个表项的初值设为一个特殊值，表示尚未填入子问题的解。当递归调用过程第一次遇到子问题时，计算其解，并存储入对应表项，随后每次遇到同一个子问题，只是简单地查表，返回其解。

##### 2.4 leetcode


| 一维  | 例题                                    | 描述      |
|-----|---------------------------------------|---------|
| 70  | Climbing Stairs                       |         |
| 62  | Unique Paths                          |         |
| 63  | Unique Paths II                       |         |
| 120 | Triangle                              | 很少考     |
| 279 | Perfect Squares                       |         |
| 139 | Word Break                            |         |
| 375 | Guess Number Higher or Lower II       |         |
| 312 | Burst Balloons                        |         |
| 322 | Coin Change                           |         |
| 二维  |                                       |         |
| 256 | Paint House                           |         |
| 265 | Paint House II                        |         |
| 64  | Minimum Path Sum                      |         |
| 72  | Edit Distance                         |         |
| 97  | Interleaving String                   |         |
| 174 | Dungeon Game                          |         |
| 221 | Maximal Square                        |         |
| 85  | Maximal Rectangle                     |         |
| 363 | Max Sum of Rectangle No Larger Than K | TreeSet |
| 化简  |                                       |         |
| 198 | House Robber                          |         |
| 213 | House Robber II                       |         |
| 276 | Paint Fence                           |         |
| 91  | Decode Ways                           |         |
| 10  | Regular Expression Matching           |         |
| 44  | Wildcard Matching                     |         |
